In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae")
model = AutoModel.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae").to('cuda')

In [2]:
# df=pd.read_csv('/kaggle/input/job-recommendation-case-study/jobs.tsv', sep='\t')
df = pd.read_csv('/kaggle/input/job-recommendation-case-study/jobs.tsv',sep = '\t',usecols=range(3,4)).astype(str)#.head(100)
df['Cleaned_Description'] = ''
df.head()
df
# del df

,Description,Cleaned_Description
0,<p>Security Clearance Required:&nbsp; Top Secr...,
1,<strong>NO Corp. to Corp resumes&nbsp;are bein...,
2,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,
3,CITY BEVERAGES Come to work for the best in th...,
4,I make sure every part of their day is magica...,
...,...,...
1091921,"<p><strong><span style=""text-decoration: under...",
1091922,"<div style=""text-align: center""><span style=""t...",
1091923,"<span>\r<hr>\r<p align=""center""><strong>Assist...",
1091924,<p>&nbsp;</p>\r<p><b><span>Macomb Community Co...,


In [ ]:
import re
from bs4 import BeautifulSoup
def cleanResume(resumeText):
    soup=BeautifulSoup(resumeText,"html.parser")
    resumeText=soup.get_text()
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
    
df['Cleaned_Description'] = df.Description.apply(lambda x: cleanResume(x))
df


In [ ]:
df['Source'] = 'KaggleManda'
df.drop(["Description"],inplace=True,axis=1)
# df = df.rename(columns={'old_name': 'new_name'})
df# assign values to new column

In [ ]:
df.to_csv("KaggleManda.csv",index=False)

In [13]:
# del df
df=pd.read_csv("/kaggle/input/scrapeddata/cleaned_jobs.csv").drop(["Unnamed: 0","Location","Company","Position"],axis=1)
# df=pd.read_csv("/kaggle/input/kagglemanda/KaggleManda_embeddings.csv").drop(["Source"],axis=1).dropna()
# df = df.rename(columns={'Cleaned_Resume': 'Cleaned_Description'})
df

,clean
0,About Accenture Accenture is a global professi...
1,Skill required Clinical SDTM Programming Desig...
2,About Accenture Accenture is a global professi...
3,Detailed JD Develop procedure and functions us...
4,About Accenture Accenture is a global professi...
...,...
823,Developer is responsible for creating Azure Da...
824,This job is sourced from a job board Learn mor...
825,Job Description The ideal candidate for this p...
826,Position Azure Data Engineer Mandatory Skills ...


In [ ]:
# df2.shape

In [ ]:
# merged_df =merged_df = pd.concat([df1, df2])
# merged_df.shape

In [ ]:
# merged_df.to_csv("Merged.csv",index=False)

In [5]:
df=pd.read_csv("/kaggle/input/merged/Merged.csv").drop(["Source"],axis=1).dropna()
df

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [40]:
df.iloc[22603]

Cleaned_Description     Responsible for reconciling an assigned Natio...
Name: 22604, dtype: object

In [14]:
list_of_lists =[item for sublist in df.values.tolist() for item in sublist]
len(list_of_lists)

828

In [ ]:
# print(len(temp))

In [19]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
import numpy as np
n=int(df.shape[0])
max_len=512
embeddings=torch.Tensor([]).to('cuda')#(np.zeros((n,df.shape[1],max_len,768)))
for i in range(n):
    sentences =list_of_lists[i]
    encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt",max_length=max_len)
#     encoded_sentences = tokenizer(sentences, return_tensors="pt")
    encoded_sentences.to('cuda')
#     print(len(encoded_sentences))
#     print(i)
#     print("encoded_sentences size=",len(encoded_sentences[0]),len(encoded_sentences['input_ids']),len(encoded_sentences['input_ids']))
    with torch.no_grad():
#         temp=[]
#         print(model(encoded_sentences['input_ids'], attention_mask=encoded_sentences['attention_mask'])[0])
        temp=model(**encoded_sentences)
#     print(model(**encoded_sentences)[0])
    sentence_embeddings = mean_pooling(temp, encoded_sentences['attention_mask'])
#         print(temp)
#     embeddings[i]=temp
#     embeddings.append(temp)
    embeddings=torch.cat((embeddings,sentence_embeddings),0)

In [20]:
print(len(embeddings))
print(len(embeddings[0]))
# print(len(embeddings[0][0]))
# print(len(embeddings[0][0][0]))

828
768


In [ ]:
print(len(embeddings[0][0][0]))

In [ ]:
embeddings

In [21]:
import pickle
# Open a file for writing
with open('jobEmbeddingArya.pickle', 'wb') as f:
    # Use pickle to dump the variable to the file
    pickle.dump(embeddings, f)

In [23]:
import pickle

# Open the pickle file for reading
with open('jobEmbeddingArya.pickle', 'rb') as f:
#     Use pickle to load the variable from the file
    embeddings = pickle.load(f)

# Print the loaded variable
print(embeddings)


tensor([[-0.1817,  0.2965, -0.0935,  ..., -0.0338,  0.3113, -0.7726],
        [-0.0982,  0.2325,  0.0284,  ...,  0.3190,  0.4740, -0.6322],
        [-0.3339,  0.2290,  0.0205,  ...,  0.2724,  0.2455, -0.5730],
        ...,
        [-0.2738,  0.6630, -0.5232,  ...,  0.0710, -0.3149, -0.2214],
        [-0.4214, -0.1128, -0.6639,  ..., -0.6259,  0.1240, -0.5204],
        [-0.1358,  0.1060, -0.7171,  ..., -0.4843,  0.2236, -0.3398]],
       device='cuda:0')


In [37]:
# list_of_lists[22603]

nan

In [36]:
# # for my_string in list_of_lists[0]:
# #         words = my_string.split()
# # len(words)
# list1=[]
# list1.append(list_of_lists[22603].split())
# # list1.append(list_of_lists[1].split())
# list1

AttributeError: 'float' object has no attribute 'split'

In [5]:
list1=[]
for i in range(len(list_of_lists)):
#         words = my_string.split()
#         print(my_string)
#     if(i>22600 and i<22700):
#         print(i)
    list1.append(list_of_lists[i].split())
len(list1)

828

In [6]:
corpus=[item for sublist in list1 for item in sublist]
len(corpus)

216857

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))
filtered_sentence = [w for w in list1 if not w.lower() in stop_words]
len(filtered_sentence)

In [17]:
from gensim.models import Word2Vec
# my_string = "This is a string with spaces"
# list1=[item for sublist in list1 for item in sublist]
# Train a Word2Vec model on the sentences
model = Word2Vec(vector_size=300,window=5,min_count=5,workers=-1)
model.build_vocab(corpus)
model.train(corpus,total_examples=model.corpus_count,epochs=5)
model.save("word2vecArya.model")
model = Word2Vec.load("word2vecArya.model")
# Get the w2v representation of a sentence
# sentence = ['this', 'is', 'sentence', 'one']
# w2v_representation = model.wv[list_of_lists[0][0]]
# for word in list1:
#   vector = model.wv[word]

# print(vector)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=25,stop_words="english")
tfidf.fit(df['clean'])
tfidf_list=dict(zip(tfidf.get_feature_names_out(),list(tfidf.idf_)))
tfidf_feature=tfidf.get_feature_names_out()

In [19]:
import numpy as np
from tqdm import tqdm
tfidf_vectors=[]
line=0
for desc in tqdm(corpus):
    sent_vec=np.zeros(300)
    weight_sum=0
    for word in desc:
        if word in model.wv.key_to_index and word in tfidf_feature:
            vec=model.wv[word]
            tf_idf=tfidf_list[word]*(desc.count(word)/len(desc))
            sent_vec+=(vec*tf_idf)
            weight_sum+=tf_idf
        if weight_sum!=0:
            sent_vec/=weight_sum
        tfidf_vectors.append(sent_vec)
        line+=1

100%|██████████| 216857/216857 [00:33<00:00, 6500.59it/s]


In [16]:
len(tfidf_vectors)

1222282

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity=cosine_similarity(tfidf_vectors,tfidf_vectors)

## Dealing with user_embedding

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae")
model = AutoModel.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae").to('cuda')

In [4]:
user_df=pd.read_csv("/kaggle/input/userresume/UserResume.csv").dropna().drop(["Resume"],axis=1)
# user_df["Combined"] = df["Category"].astype(str) + df["Cleaned_Resume"]
user_df.head()

,Category,Cleaned_Resume
0,Data Science,Skills Programming Languages Python pandas num...
1,Data Science,Education Details May 2013 to May 2017 B E UIT...
2,Data Science,Areas of Interest Deep Learning Control System...
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,Data Science,Education Details MCA YMCAUST Faridabad Haryan...


In [5]:
user_df["Combined"] = user_df["Category"].astype(str) + user_df["Cleaned_Resume"]
user_df=user_df.drop(["Category","Cleaned_Resume"],axis=1)
user_df

,Combined
0,Data ScienceSkills Programming Languages Pytho...
1,Data ScienceEducation Details May 2013 to May ...
2,Data ScienceAreas of Interest Deep Learning Co...
3,Data ScienceSkills R Python SAP HANA Tableau S...
4,Data ScienceEducation Details MCA YMCAUST Fari...
...,...
957,TestingComputer Skills Proficient in MS office...
958,Testing Willingness to a ept the challenges Po...
959,TestingPERSONAL SKILLS Quick learner Eagerness...
960,TestingCOMPUTER SKILLS SOFTWARE KNOWLEDGE MS P...


In [6]:
list_of_lists =[item for sublist in user_df.values.tolist() for item in sublist]
len(list_of_lists)

962

In [7]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
import numpy as np
n=int(user_df.shape[0])
max_len=512
embeddings=torch.Tensor([]).to('cuda')#(np.zeros((n,df.shape[1],max_len,768)))
for i in range(n):
    sentences =list_of_lists[i]
    encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt",max_length=max_len)
    encoded_sentences.to('cuda')
    with torch.no_grad():
        temp=model(**encoded_sentences)
    sentence_embeddings = mean_pooling(temp, encoded_sentences['attention_mask'])
    embeddings=torch.cat((embeddings,sentence_embeddings),0)

In [11]:
import pickle
# Open a file for writing
with open('userEmbeddingArya.pickle', 'wb') as f:
    # Use pickle to dump the variable to the file
    pickle.dump(embeddings, f)

In [12]:
import pickle

# Open the pickle file for reading
with open('userEmbeddingArya.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    embeddings = pickle.load(f)

# Print the loaded variable
print(len(embeddings),len(embeddings[0]))


962 768


## Stacking user and job embeddings

In [24]:
import pickle

# Open the pickle file for reading
with open('/kaggle/input/embeddings/userEmbeddingArya.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    user_embeddings = pickle.load(f)

with open('/kaggle/input/embeddings/jobEmbeddingArya.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    job_embeddings = pickle.load(f)
    
# Print the loaded variable
print(len(user_embeddings),len(user_embeddings[0]))
print(len(job_embeddings),len(job_embeddings[0]))

962 768
828 768


In [26]:
import itertools
import pandas as pd

# Create two example tensors
tensor1 = user_embeddings
tensor2 = job_embeddings

# Generate all permutations of the two tensors
permutations = itertools.product(tensor1, tensor2)

# Convert tuples to rows in a DataFrame
Embedding_df = pd.DataFrame(permutations, columns=['tensor1', 'tensor2'])

# Display the resulting DataFrame
print(Embedding_df.shape)


(796536, 2)


In [31]:
import torch.nn.functional as F
Embedding_df['similarity'] = F.cosine_similarity(Embedding_df['tensor1'], Embedding_df['tensor2'])

TypeError: cosine_similarity(): argument 'x1' (position 1) must be Tensor, not Series